# Les C7: Time series forecasting  
In deze les zien we hoe je _time series forecasting_ aanpakt. Dit doen we aan de hand van ARIMA-modellen.

## Benodigde libraries, functies en data  
Zoals altijd beginnen we met het importeren van het een en ander. Allereerst de libraries.

In [17]:
# importeer libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima.model import ARIMA
import seaborn as sns
sns.set_style('whitegrid')

Hieronder volgt een aantal code chunks met functies die specifiek voor de situaties in deze notebook goed werken. **Houd er rekening mee dat ze dus niet in alle situaties goed hoeven te werken!**

In [2]:
# functie voor plotten van time series
def plot_series(df, series_name, lags=40, diff=0, seasonal_diff=0, seasonal_period=12):
    """
    Plot a time series alongside its ACF and PACF with Bartlett bounds.
    Includes options for regular and seasonal differencing.
    
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing multiple time series as columns.
    series_name : str
        Column name of the series to plot.
    lags : int
        Number of lags for ACF/PACF.
    diff : int, default=0
        Number of regular differences to apply.
    seasonal_diff : int, default=0
        Number of seasonal differences to apply.
    seasonal_period : int, default=12
        Seasonal period (e.g., 12 for monthly data with yearly seasonality).
    """
    series = df[series_name]

    # Apply differencing
    for _ in range(diff):
        series = series.diff()
    for _ in range(seasonal_diff):
        series = series.diff(seasonal_period)
    series = series.dropna()

    # Compute ACF/PACF (drop lag 0)
    acf_vals = acf(series, nlags=lags, fft=False)[1:]
    pacf_vals = pacf(series, nlags=lags, method="ywm")[1:]
    lags_range = np.arange(1, len(acf_vals)+1)

    # Bartlett bounds
    n = len(series)
    conf = 1.96 / np.sqrt(n)

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    # Time series
    axes[0].plot(series.index, series.values, color="steelblue")
    axes[0].set_title(f"Series: {series_name} (diff={diff}, seas_diff={seasonal_diff})")

    # ACF
    axes[1].stem(lags_range, acf_vals, basefmt=" ")
    axes[1].hlines([conf, -conf], xmin=0, xmax=lags, colors="red", linestyles="dashed")
    axes[1].axhline(0, color="black", linewidth=0.8)
    axes[1].set_title("ACF-plot")

    # PACF
    axes[2].stem(lags_range, pacf_vals, basefmt=" ")
    axes[2].hlines([conf, -conf], xmin=0, xmax=lags, colors="red", linestyles="dashed")
    axes[2].axhline(0, color="black", linewidth=0.8)
    axes[2].set_title("PACF-plot")

    plt.tight_layout()
    plt.show()

In [3]:
# functie voor maken van time series cross-validation splits
def make_time_series_splits(series, initial_window, horizon=1, step=1, window_type="expanding"):
    """
    Generate time-series cross-validation splits.

    Parameters
    ----------
    series : pd.Series
        Time series (DateTimeIndex or RangeIndex).
    initial_window : int
        Number of observations in the first training window.
    horizon : int
        Forecast horizon (steps ahead).
    step : int
        How many steps to move the origin each iteration.
    window_type : str
        "expanding"  -> training window grows over time
        "sliding"    -> training window has fixed size

    Returns
    -------
    splits : list of tuples
        Each tuple is (train_idx, test_idx), where each is an array of positions.
    """
    n = len(series)
    splits = []

    start = initial_window

    while start + horizon <= n:
        if window_type == "expanding":
            # Training window always starts at 0
            train_idx = np.arange(0, start)

        elif window_type == "sliding":
            # Fixed-size training window
            train_idx = np.arange(start - initial_window, start)

        else:
            raise ValueError("window_type must be 'expanding' or 'sliding'")

        test_idx = np.arange(start, start + horizon)
        splits.append((train_idx, test_idx))

        start += step

    return splits

In [ ]:
# gebruikelijke evaluation metrics voor time series forecasting
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def mape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_true))
    )

In [ ]:
# functies voor time series cross-validation
def cv_single_arima(series, order, seasonal_order, splits, metric=rmse):
    """
    Evaluate one ARIMA specification across CV splits.
    
    Returns the average error across folds.
    """
    errors = []

    for train_idx, test_idx in splits:
        train = series.iloc[train_idx]
        test = series.iloc[test_idx]

        model = ARIMA(train, order=order, seasonal_order=seasonal_order)
        fitted = model.fit()

        fc = fitted.forecast(steps=len(test))
        error = metric(test.values, fc.values)
        errors.append(error)

    return np.mean(errors)

def cv_arima_candidates(series, candidates, splits, metric=rmse):
    """
    Evaluate multiple ARIMA candidates.
    
    Parameters
    ----------
    candidates : list of dicts
        Example:
        [
            {"order": (1,1,1), "seasonal_order": (0,1,1,12)},
            {"order": (2,1,0), "seasonal_order": (1,1,0,12)}
        ]
    """
    results = []

    for spec in candidates:
        avg_error = cv_single_arima(
            series,
            order=spec["order"],
            seasonal_order=spec["seasonal_order"],
            splits=splits,
            metric=metric
        )
        results.append({
            "order": spec["order"],
            "seasonal_order": spec["seasonal_order"],
            "cv_error": avg_error
        })

    return pd.DataFrame(results)

In [ ]:
# functies voor visualiseren fouten
def horizon_errors(series, order, seasonal_order, H, splits, metric):
    """
    Compute horizon-wise forecast errors for ARIMA using any CV splits.
    
    Parameters
    ----------
    series : pd.Series
    order : tuple
    seasonal_order : tuple
    H : int
        Maximum forecast horizon.
    splits : list of (train_idx, test_idx)
        Output of make_time_series_splits (expanding or sliding).
    """
    errors = {h: [] for h in range(1, H+1)}

    for train_idx, test_idx in splits:
        train = series.iloc[train_idx]
        test = series.iloc[test_idx]

        model = ARIMA(train, order=order, seasonal_order=seasonal_order)
        fitted = model.fit()

        fc = fitted.forecast(steps=H)

        for h in range(1, H+1): 
            e = metric(test.iloc[h-1], fc.iloc[h-1]) 
            errors[h].append(e)

    avg_errors = {h: np.mean(errors[h]) for h in errors}
    return avg_errors, errors

def plot_horizon_boxplot(all_errors):
    """
    Boxplot of horizon-wise forecast errors.
    """
    horizons = list(all_errors.keys())
    data = [all_errors[h] for h in horizons]

    plt.figure(figsize=(10, 5))
    plt.boxplot(data, labels=horizons, showfliers=True)
    plt.xlabel("Forecast horizon (steps ahead)")
    plt.ylabel("")
    plt.title("Forecast error distribution")
    plt.grid(axis='y', alpha=0.3)
    plt.show()

Tenslotte nog wat code waarmee we de dataset inlezen. Hierbij gaan we ervan uit dat je `nl_airports.csv` op de juiste plaats hebt gezet, zodat Python dat bestand kan vinden.

In [ ]:
# dataset inlezen
# let op het scheidingsteken
nl_airports = pd.read_csv('nl_airports.csv', sep = ';')

# maak van Month een datetime-object
nl_airports['Month'] = pd.to_datetime(nl_airports['Month'], format="%Y %m")

# hernoem kolommen van vliegvelden
nl_airports = nl_airports.rename(columns={"Amsterdam Airport Schiphol": "AMS", "Rotterdam The Hague Airport": "RTM", "Eindhoven Airport": "EIN", "Maastricht Aachen Airport": "MST", "Groningen Airport Eelde": "GRQ"})

# maak van Month de index en sorteer deze
nl_airports = nl_airports.set_index('Month').sort_index()

# geef de frequentie van de index aan
nl_airports = nl_airports.asfreq("MS")

## Selecteren van kandidaatmodellen  
We richten ons op de passagiersaantallen op Schiphol, in deze dataset nu aangeduid met `AMS`. In de onderstaande code zie je dat een ARIMA(0,1,0)(0,1,1)$_{12}$ is gefit op deze time series.

In [ ]:
# fit SARIMA-model met (p,d,q)=(0,1,0), (P,D,Q)=(0,1,1) en m=12
model = ARIMA(nl_airports["AMS"], order=(0,1,0), seasonal_order=(0,1,1,12))
fitted = model.fit()

# plot gefitte ARIMA-model met de time series
plt.plot(nl_airports.index, fitted.fittedvalues, label="ARIMA(0,1,0)(0,1,1)[12]", color="darkorange")
plt.plot(nl_airports.index, nl_airports['AMS'], label="AMS", color="steelblue")
plt.title("AMS and fitted SARIMA-model")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.show()

**_Opgave 1_**  
Run bovenstaande code en ga na dat het model in het begin nog veel moeite heeft met het goed fitten op de time series. Heb je daar een verklaring voor?

**_Opgave 2_**  
Vind je ARIMA(0,1,0)(0,1,1)$_{12}$ een goede keuze aan parameters voor deze time series?  
Zo ja, waarom? Hoe kan je dat nagaan?  
Zo nee, waarom niet? Wat zouden volgens jou wel goede parameters zijn?

In les C5 en C6 hebben we gezien dat het selecteren van parameters voor ARIMA-modellen deels een subjectieve gelegenheid is. Dat betekent dat er niet een exacte manier is om van tevoren na te gaan of je de "beste" combinatie van parameters (wat dat dan ook betekent) hebt gevonden. Voor voorspellingsdoeleinden zou je daarentegen juist wel graag die zekerheid willen. Maar omdat dat niet van tevoren kan, kunnen we dat alleen maar achteraf nagaan.  

In theorie zou je dus alle mogelijke parametercombinaties kunnen afgaan, net zolang tot je de beste gevonden hebt. Zo'n uitputtende grid search werkt zeker, maar leidt al heel snel tot overfitte modellen (zelfs met de nodige regularisatie). Om die reden heeft een uitputtende grid search voor ARIMA niet de voorkeur.  

In plaats daarvan kiezen we voor een slimmere grid search, eentje die een stuk meer afgebakend is. Deze werkt als volgt:  
1. Kies eerst één combinatie van geschikte parameterwaarden $(p,d,q)(P,D,Q)_m$ aan de hand van (P)ACF-plots.  
2. Maak vanuit deze combinatie meerdere mogelijke combinaties door de waarden van $p,q,P,Q$ allemaal met $1$ te verhogen en te verlagen. Zodoende krijg je nu maximaal 81 kandidaatmodellen.  
3. Verwijder alle kandidaatmodellen waarbij minstens één van $p,q,P,Q$ negatief is, en ook waarbij $p+q$ en/of $P+Q$ groot zijn.  

Na stap 3 houd je een kleinere selectie aan combinaties over. Dit zijn de kandidaatmodellen waar je verder mee aan de slag gaat.

**_Opgave 3_**  
Ga uit van de parametercombinatie die je bij Opgave 2 hebt gekozen. Bepaal vanuit deze combinatie de kandidaatmodellen waar jij mee verder wilt gaan. Mik in eerste instantie op **niet meer dan 10** kandidaatmodellen.

## Time series forecasting
### Waarom time series cross-validation anders is  
Het idee van voorspellen en 'probeer veel modellen en bekijk daarna welke het beste presteert', doet je wellicht denken aan data science. En inderdaad, we gaan hier ook een data science-aanpak hanteren. Om precies te zijn, het principe van _cross-validation_: we houden data apart achter waar het model niet op traint, zodat we later kunnen nagaan of het model inderdaad goed in staat is om te voorspellen op nieuwe data.  

Toch gaat cross-validation niet op de standaard manier zoals je die kent van data science. En het verschil komt door de index: er zit namelijk een volgorde in de tijd. _Forecasting_ kan je zien als het voorspellen van de time series voor toekomstige waarden van de index. Dat betekent dat we dat aspect - het voorspellen bij toekomstige index-waarden - ook moeten nabootsen in cross-validation. En dat lukt niet met een _random shuffle_, zoals dat gebruikelijk is. Voor een _time series cross-validation_ geldt dat de test set altijd chronologisch na de training set komt en nooit andersom.  

### Trainen en testen: time series jargon  
In de wereld van _time series forecasting_ bezigen we specifieke jargon, waarvan het goed is dat je die kent.  
* De _horizon_ is de lengte van de periode die je probeert te voorspellen. Als je een tijdreeks op maandbasis hebt en je wilt 1 jaar vooruit voorspellen, dan is je horizon dus 12 maanden.  
* De _test set_ is een gedeelte van de waargenomen time series dat gebruikt wordt om de prestaties van het voorspellingsmodel mee te meten. In de regel is de even groot als de horizon.  
* De _training set_ is een gedeelte van de waargenomen time series dat gebruikt wordt om het voorspellingsmodel op te trainen. De training set zit, chronologisch gezien, altijd voorafgaand aan de test set. Verder bevat de training set minstens 2 keer zo veel datapunten als groottes van time series componenten die je in het model wilt meenemen. Als je time series een seasonality met periode 7 heeft, moet je training set uit minstens 14 datapunten bestaan om het model de seasonality te laten leren.

**_Opgave 4_**  
Stel, je wilt een ARIMA-model gebruiken om een time series te forecasten. De time series bevat een trend en een seasonality met periode 16. Je hebt aan twee keer regular differencing en één keer seasonal differencing genoeg om de time series stationary te maken. De beoogde horizon is 8 tijdseenheden.  
Toon aan dat je dan minstens 44 datapunten moet hebben om time series forecasting te kunnen uitvoeren.

In Opgave 4 heb je eigenlijk alleen maar gekeken naar het absolute minimum aan datapunten zodat het technisch gezien mogelijk is om time series forecasting te kunnen doen. Maar dat maakt het nog zeker geen goede forecasting; daarvoor wil je toch echt wel meer datapunten hebben. Hoeveel meer, daar zijn geen harde regels voor; dat hangt namelijk ook heel sterk af van de context (wat stelt de time series in de praktijk voor). Een grove vuistregel (maar dus zeker geen exacte regel) is dat je minstens 5 seasonalities in je training set wilt hebben zitten.

**_Opgave 5_**  
Voor de passagiersaantallen op Schiphol willen we een voorspelling maken voor 1 jaar vooruit.  
Hebben we genoeg data voor een potentieel goede time series forecasting? Waarom?  
Hoe groot moet de training set zijn? En de test set?

## Time series cross-validation  
Time series cross-validation is dus, zoals eerder benoemd, niet hetzelfde als de gewone cross-validation (CV). Net als bij gewone cross-validation willen we meerdere train-testsplits kunnen maken, om zo een beter beeld te krijgen van de prestaties van het model. Het gebruik van meerdere splits maakt de kans dat er in onze resultaten een _bias_ zit, minder groot. We moeten wel goed nadenken hoe we aan die meerdere splits komen, aangezien we wel de chronologische volgorde (eerst training, daarna test) steeds in stand moeten houden. Grofweg zijn er twee varianten van time series cross-validation:  
* _rolling window cross-validation_ (ook wel _expanding window cross-validation_ genoemd): In de eerste fold bestaat de training set uit de eerste waarnemingen van de time series, zo groot als nodig is. Daarachter zit de test set geplakt, even groot als de horizon. In elk daaropvolgende fold wordt de training set uitgebreid met $k$ tijdstappen, en schuift de test set met $k$ tijdstappen op. Hierdoor geldt in elke fold dat de testset steeds direct volgt op de training set.  
* _sliding window cross-validation_: De eerste fold is identiek aan die bij rolling window cross-validation. Maar in elke daaropvolgende fold schuift de training set in zijn geheel op met $k$ tijdstappen (en de test set dus ook). Het verschil is dat hier de training set altijd dezelfde grootte heeft, ongeacht naar welke fold je kijkt.  

Beide varianten worden in de praktijk gebruikt en er lijkt geen consensus te zijn over welke variant beter is. Rolling window CV is wat natuurlijker, in die zin dat in de praktijk vaak zoveel mogelijk data gebruikt wordt voor voorspellingsdoeleinden. Het idee van groeiende training sets sluit daar beter op aan. Sliding window CV is daarentegen weer wat interessanter op het moment dat je teveel data kan hebben (bijvoorbeeld omdat het trainingsproces dan te lang duurt). Sliding window CV maakt het ook wat makkelijker/eerlijker om verschillende folds met elkaar te vergelijken en zodoende na te gaan of modellen moeite hebben met specifieke stukken van de time series.

**_Opgave 6_**  
Voor de passagiersaantallen op Schiphol willen we een voorspelling maken voor 1 jaar vooruit.  
Gebruik de functie `make_time_series_splits()` om de splits voor een rolling window cross-validation te maken, waarbij je de training set steeds met 5 maanden uitbreidt.  
Kijk goed naar de functiebeschrijving bovenaan deze notebook, om te zien welke argumenten de functie heeft. En vergeet je antwoord bij Opgave 5 niet!

**_Opgave 7_**  
Wat gaat er mis als je in Opgave 6 de training set zou uitbreiden met 12 maanden?  
Zou dat wel goed gaan bij 24 maanden? En bij 3 maanden?

**_Opgave 8_**  
Zet de door jou bij Opgave 3 gekozen kandidaatmodellen in een dictionary. Hieronder is een beginnetje gemaakt.

In [ ]:
# dictionary van kandidaatmodellen
candidates = [{"order": (0,1,0), "seasonal_order": (0,1,1,12)}, 
              ...]

**_Opgave 9_**  
Voer de rolling window CV uit, met behulp van jouw antwoorden bij Opgave 6, Opgave 8 en de functie `cv_arima_candidates()`.  
Welk model scoort het beste aan de hand van de MAPE?

**_Opgave 10_**  
Plot de voorspelling voor het jaar 2019 volgens het model dat bij Opgave 9 het beste was.

### Hoe goed is het model nu echt?  
Uit Opgave 9 komt weliswaar het beste van de kandidaatmodellen naar voren, maar daaruit maken we nog niet op hoe goed dat model nu echt in staat is om te voorspellen. Een logische gedachte is dat hoe verder vooruit je probeert te voorspellen, hoe groter de fouten zullen zijn. Immers, je stapelt onzekerheid op onzekerheid, waardoor fouten ook potentieel groter kunnen worden. De meeste evaluation metrics als RMSE, MAE en MAPE verhullen dat effect, omdat ze het gemiddelde van alle fouten over de hele horizon nemen.

Een visualisatie waarmee je de groei van de voorspellingsfouten in de horizon kan laten zien, wordt ook wel de _forecast error distribution-plot_ genoemd. Er zijn meerdere manieren om zo'n plot weer te geven. De functie `plot_horizon_boxplot()` doet dit aan de hand van boxplots per tijdstap in de horizon. De keuze voor boxplots is dat je dan ook nog onderscheid kan maken tussen toevallige uitschieters (die wellicht door _time series outliers_ zijn veroorzaakt) en systematische foutengroei.

**_Opgave 11_**  
Maak een forecast error distribution-plot bij het model dat bij Opgave 9 het beste was. Gebruik de functies `horizon_errors()` en `plot_horizon_boxplot()`.

## Waarom ARIMA?  
Voor een heel groot deel is de methodiek van deze les niet afhankelijk van het feit dat we met ARIMA-modellen werken. Dat betekent dat je dezelfde werkwijze ook kan gebruiken voor andere soorten modellen. Met de nodige _feature-engineering_ kan je ook de modellen van DS2 en DS3 gebruiken voor time series forecasting.  

Achter het gebruik van ARIMA-modellen schuilt wel een centrale aanname: je neemt aan dat algemene kenmerken van de time series (namelijk, de autocorrelaties) ook representatief zijn voor de nabije toekomst. Maar dat hoeft natuurlijk niet zo te zijn. Zeker bij time series die sterk afhankelijk zijn van externe invloeden, geldt dat data science-modellen daar beter mee kunnen omgaan; die werken namelijk niet vanuit de centrale aanname waar ARIMA op gestoeld is.